In [1]:
import sys
import logging

import numpy as np
import matplotlib.pyplot as plt
from joblib import hash, dump, load
import os

from deer.default_parser import process_args
from deer.agent import NeuralAgent
from deer.learning_algos.CRAR_torch import CRAR
from figure8_env import MyEnv as figure8_env
import deer.experiment.base_controllers as bc

from deer.policies import EpsilonGreedyPolicy, FixedFigure8Policy

In [2]:
figure8_give_rewards = True
nn_yaml = 'network_noconv.yaml'
higher_dim_obs = False
internal_dim = 10
fname = 'test'

In [3]:
class Defaults:
    # ----------------------
    # Experiment Parameters
    # ----------------------
    steps_per_epoch = 5000
    epochs = 50
    steps_per_test = 1000
    period_btw_summary_perfs = 1

    # ----------------------
    # Temporal Processing Parameters
    # ----------------------
    nstep = 20
    nstep_decay = 0.8
    encoder_type = 'variational'
    
    # ----------------------
    # Environment Parameters
    # ----------------------
    frame_skip = 2
    show_rewards = False

    # ----------------------
    # DQN Agent parameters:
    # ----------------------
    learning_rate = 1 * 1E-4
    learning_rate_decay = 1.0
    discount = 0.9
    epsilon_start = 1.0
    epsilon_min = 1.0
    epsilon_decay = 1000
    update_frequency = 1
    replay_memory_size = 100000 #50000
    batch_size = 64
    freeze_interval = 1000
    deterministic = False
    
    # ----------------------
    # Learning algo parameters
    # ----------------------
    #loss_weights = [5E-3, 1E-3, 5E-3, 5E-3, 5E-3, 5E-3, 1.]
    #loss_weights = [0, 0, 0, 0, 0, 0, 1.]
    loss_weights = [5E-3, 5E-3, 5E-3, 0, 5E-3, 1E-3, 1., 1E-6]

In [4]:
parameters = Defaults()
if parameters.deterministic:
    rng = np.random.RandomState(123456)
else:
    rng = np.random.RandomState()

# --- Instantiate environment ---
env = figure8_env(
    give_rewards=figure8_give_rewards,
    intern_dim=internal_dim,
    higher_dim_obs=higher_dim_obs,
    show_rewards=parameters.show_rewards,
    nstep=parameters.nstep, nstep_decay=parameters.nstep_decay
    )

# --- Instantiate learning_algo ---
learning_algo = CRAR(
    env,
    parameters.freeze_interval,
    parameters.batch_size,
    rng,
    high_int_dim=False,
    internal_dim=internal_dim, lr=parameters.learning_rate,
    nn_yaml=nn_yaml, double_Q=True,
    loss_weights=parameters.loss_weights,
    nstep=parameters.nstep, nstep_decay=parameters.nstep_decay,
    encoder_type=parameters.encoder_type
    )

if figure8_give_rewards:
    train_policy = EpsilonGreedyPolicy(
        learning_algo, env.nActions(), rng, 0.2,
        consider_valid_transitions=False
        )
    test_policy = EpsilonGreedyPolicy(
        learning_algo, env.nActions(), rng, 0.
        )
else:
    train_policy = FixedFigure8Policy.FixedFigure8Policy(
        learning_algo, env.nActions(), rng, epsilon=0.2,
        height=env.HEIGHT, width=env.WIDTH
        )
    test_policy = FixedFigure8Policy.FixedFigure8Policy(
        learning_algo, env.nActions(), rng,
        height=env.HEIGHT, width=env.WIDTH
        )

# --- Instantiate agent ---
agent = NeuralAgent(
    env, learning_algo,
    parameters.replay_memory_size,
    1, parameters.batch_size, rng,
    train_policy=train_policy, test_policy=test_policy)

agent.run(10, 500)
print("end gathering data")

# --- Bind controllers to the agent ---
# Before every training epoch (periodicity=1), we want to print a summary of the agent's epsilon, discount and 
# learning rate as well as the training epoch number.
agent.attach(bc.VerboseController(
    evaluate_on='epoch', 
    periodicity=1))

# Learning rate may follow a scheduler
agent.attach(bc.LearningRateController(
    initial_learning_rate=parameters.learning_rate, 
    learning_rate_decay=parameters.learning_rate_decay,
    periodicity=1))

# During training epochs, we want to train the agent after every [parameters.update_frequency] action it takes.
# Plus, we also want to display after each training episode (!= than after every training) the average bellman
# residual and the average of the V values obtained during the last episode, hence the two last arguments.
agent.attach(bc.TrainerController(
    evaluate_on='action', 
    periodicity=parameters.update_frequency, 
    show_episode_avg_V_value=True, 
    show_avg_Bellman_residual=True))

# We wish to discover, among all versions of our neural network (i.e., after every training epoch), which one 
# has the highest validation score.
# To achieve this goal, one can use the FindBestController along with an InterleavedTestEpochControllers. It is 
# important that the validationID is the same than the id argument of the InterleavedTestEpochController.
# The FindBestController will dump on disk the validation scores for each and every network, as well as the 
# structure of the neural network having the best validation score. These dumps can then used to plot the evolution 
# of the validation and test scores (see below) or simply recover the resulting neural network for your 
# application.
agent.attach(bc.FindBestController(
    validationID=figure8_env.VALIDATION_MODE,
    testID=None,
    unique_fname=fname))

# All previous controllers control the agent during the epochs it goes through. However, we want to interleave a 
# "validation epoch" between each training epoch. For each validation epoch, we want also to display the sum of all 
# rewards obtained, hence the showScore=True. Finally, we want to call the summarizePerformance method of ALE_env 
# every [parameters.period_btw_summary_perfs] *validation* epochs.
agent.attach(bc.InterleavedTestEpochController(
    id=figure8_env.VALIDATION_MODE, 
    epoch_length=parameters.steps_per_test,
    periodicity=1,
    show_score=True,
    summarize_every=1,
    unique_fname=fname))

end gathering data


In [5]:
try:
    os.mkdir("params")
except Exception:
    pass
dump(vars(parameters), "params/" + fname + ".jldump")
#agent.gathering_data=False
agent.run(parameters.epochs, parameters.steps_per_epoch)

# --- Show results ---
basename = "scores/" + fname
scores = load(basename + "_scores.jldump")
print(scores)

Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([-0.9788, -1.2043, -1.1370, -1.3863,  0.8605,  1.9189, -1.1895, -1.0534,
        -0.1351,  0.6652]) tensor([-0.7933, -1.1102, -1.4306, -1.6623,  0.9538,  2.0401, -0.9203, -1.2591,
         0.0874,  0.8875]) tensor([-0.9202, -0.5225,  0.6498,  0.9199, -1.9167,  0.5375,  0.9473,  0.4975,
         0.7661, -0.2791])
R[0]
tensor([0.2181], grad_fn=<SelectBackward0>)
LOSSES
T = 1.2190736744403838; R = 0.007600422054587398;                 Gamma = 0.3480693909712136; Q = 0.025976787250488995;
Entropy Neighbor = 1.1845786672424196e-05;                 Entropy Random = 1.0816577540484928e-05;                 Volume = 2.816436332702637; VAE = 47.36184683227539
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
1 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.8675,  1.4538, -0.6874, -0.3086, -1.1682,  0.6896,  1.1189, -0.0783,

/Users/chingfang/miniforge3/envs/auxrl/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Testing score per episode (id: 0) is 0.0 (average over 1 episode(s))
== Mean score per episode is 0.0 over 1 episodes ==


/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:275: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  abs_states[i:i+1], torch.as_tensor([action_encoding])
/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:301: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(
/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:333: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
1 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 1.4236, -0.2033, -0.4562, -0.7022,  0.0916,  0.3257,  0.7770, -2.1177,
        -0.0353,  0.9046]) tensor([ 1.4265, -0.4433, -0.3746, -0.6656,  0.3548,  0.3689,  0.7101, -2.0564,
        -0.1040,  0.8071]) tensor([ 1.7488, -0.2470, -0.6419, -0.7277,  0.4187,  0.4013,  1.1190, -2.2989,
        -0.1961,  1.0042])
R[0]
tensor([-4.6074e-05], grad_fn=<SelectBackward0>)
LOSSES
T = 0.06847252438962459; R = 0.0009049177863053046;                 Gamma = 0.016333105246245396; Q = 0.0007320378847653046;
Entropy Neighbor = 0.014011786185204983;                 Entropy Random = 0.008479813725687563;                 Volume = 2.2956529932022094; VAE = 47.107276153564456
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
3 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 1.3439, -0.5752, -1.0429, -0.0489,  0.2629,  0.3068,  1.2532, 

...

/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:301: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(
/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:333: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([ 0.6412, -0.2374, -0.6108, -0.5812,  1.0468, -0.0178,  0.9238, -1.0989,
         0.1077, -0.1351]) tensor([ 0.5065, -0.3522, -0.6920, -0.6156,  0.5066,  0.3536,  0.9003, -1.0453,
         0.1180, -0.0942]) tensor([ 0.6246, -0.6303, -0.1820, -1.0062,  0.5504,  0.3437,  0.9195, -1.0773,
         0.1464,  0.3611])
R[0]
tensor([0.0245], grad_fn=<SelectBackward0>)
LOSSES
T = 0.07043951278179884; R = 0.012693849506904371;                 Gamma = 0.0057332254391876634; Q = 0.00920137558504939;
Entropy Neighbor = 0.030767518252134324;                 Entropy Random = 0.00447754874243401;                 Volume = 0.204677266061306; VAE = 28.087453285217286
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
1 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([-0.9276,  0.0393, -0.7770, -0.1600,  0.2643,  1.3971,  0.5205, -0.3234,


/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:301: UserWarning: You passed a edgecolor/edgecolors ('k') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(
/Users/chingfang/Code/deer/examples/test_CRAR/figure8_env.py:333: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
0 1.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([-1.3791,  0.2224,  1.0298,  0.2643, -0.1397, -0.1387, -0.8451,  0.8435,
         0.6960, -0.0820]) tensor([-0.4569,  0.0057,  0.0988, -0.0783,  0.1059,  0.2838, -0.1656, -0.0409,
         0.4625,  0.0502]) tensor([ 0.2162, -0.2376, -0.9401, -0.7658,  0.3711,  0.4971,  1.0527, -1.0136,
        -0.0428,  0.2527])
R[0]
tensor([0.8819], grad_fn=<SelectBackward0>)
LOSSES
T = 0.06779227401316167; R = 0.015141792837472167;                 Gamma = 0.0058837916517077244; Q = 0.010271765822079032;
Entropy Neighbor = 0.032048573043197394;                 Entropy Random = 0.004483509021927602;                 Volume = 0.2181043034940958; VAE = 28.929636981964112
Printing a few elements useful for debugging:
actions_val[0], rewards_val[0], terminals_val[0]
1 0.0 0.0
Es[0], TEs[0], Esp_[0]
tensor([-0.4459, -0.4440, -0.6618, -0.7296, -0.1697,  0.7260,  0.9813, -0.804


KeyboardInterrupt



In [ ]:
x = learning_algo.get_losses()[0]

In [ ]:
x[-1]

## Visualize performance

In [ ]:
agent.setNetwork(fname, nEpoch=33)

In [ ]:
agent._in_episode = True
agent._mode = 0 # Testing mode with plan_depth=0
initState = env.reset(agent._mode)
inputDims = env.inputDimensions()

for i in range(len(inputDims)):
    if inputDims[i][0] > 1:
        agent._state[i][1:] = initState[i][1:]
agent._Vs_on_last_episode = []
is_terminal = False
reward = 0

### Frame by frame

In [ ]:
%matplotlib inline

for i in range(100):
    obs = env.observe()
    _obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
    plt.figure()
    plt.imshow(np.flip(_obs.squeeze()))
    plt.show()
    for i in range(len(obs)):
        agent._state[i][0:-1] = agent._state[i][1:]
        agent._state[i][-1] = obs[i]
    V, action, reward, _ = agent._step()
    print(action)
    agent._Vs_on_last_episode.append(V)
    is_terminal = env.inTerminalState()
    if is_terminal: break

### As animation

In [ ]:
%matplotlib notebook

import numpy as np 
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

fig, ax = plt.subplots(1,1)
obs = env.observe()
_obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
_obs = np.flip(_obs.squeeze())
ax.set_xticks([])
ax.set_yticks([])
im = ax.imshow(np.zeros(_obs.shape))

def init():
    plt.cla()
    im = ax.imshow(_obs)
    return [im]

def animate(i, *args, **kwargs):
    plt.cla()
    obs = env.observe()
    _obs = obs[0].reshape((env.WIDTH, env.HEIGHT))
    _obs = np.flip(_obs.squeeze())
    im = ax.imshow(_obs)
    for i in range(len(obs)):
        agent._state[i][0:-1] = agent._state[i][1:]
        agent._state[i][-1] = obs[i]
        V, action, reward, _ = agent._step()
        agent._Vs_on_last_episode.append(V)
    return [im]

ani = animation.FuncAnimation(fig, animate, init_func=init, 
     frames=100, blit=False, repeat=True)
ani.save('behavior.gif', writer="ffmpeg", fps = 15)
fig.show()